# ОИАД. Лабораторная работа №1

**Студент:** Воробей Артём Владимирович  
**Группа:** 13-2025  
**Номер в списке группы (N):** 2  
**Столбец для анализа:** Exercise_Hours (N % 7 = 2 % 7 = 2)

Набор данных: `teen_phone_addiction_dataset.csv`


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chi2, skew, kurtosis
import warnings
warnings.filterwarnings('ignore')

# Настройка отображения графиков
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10


In [ ]:
# Загрузка данных
df = pd.read_csv('../../datasets/teen_phone_addiction_dataset.csv')

# Определение столбца для анализа
N = 2  # Номер в списке группы
cols = ['Daily_Usage_Hours', 'Sleep_Hours', 'Exercise_Hours', 'Screen_Time_Before_Bed', 
        'Time_on_Social_Media', 'Time_on_Gaming', 'Time_on_Education']
column_name = cols[N % 7]

print(f"Номер в списке группы: {N}")
print(f"Анализируемый столбец: {column_name}")
print(f"\nПервые 5 строк данных:")
df.head()


In [ ]:
# Извлечение данных из выбранного столбца
data = df[column_name].values

print(f"Информация о столбце '{column_name}':")
print(f"Количество наблюдений: {len(data)}")
print(f"Минимальное значение: {data.min():.4f}")
print(f"Максимальное значение: {data.max():.4f}")
print(f"Первые 20 значений: {data[:20]}")


## I. Расчет характеристик и построение графиков


In [ ]:
# Функция для расчета всех статистических характеристик
def calculate_statistics(data):
    """
    Расчет статистических характеристик выборки
    """
    stats_dict = {}
    
    # 1. Среднее
    stats_dict['Среднее'] = np.mean(data)
    
    # 2. Дисперсия
    stats_dict['Дисперсия'] = np.var(data, ddof=1)  # ddof=1 для несмещенной оценки
    
    # 3. Мода
    from scipy import stats as sp_stats
    mode_result = sp_stats.mode(data, keepdims=True)
    stats_dict['Мода'] = mode_result.mode[0]
    
    # 4. Медиана
    stats_dict['Медиана'] = np.median(data)
    
    # 5. Квантили
    stats_dict['Квантиль 0.25'] = np.quantile(data, 0.25)
    stats_dict['Квантиль 0.5'] = np.quantile(data, 0.5)
    stats_dict['Квантиль 0.75'] = np.quantile(data, 0.75)
    
    # 6. Эксцесс
    stats_dict['Эксцесс'] = kurtosis(data, fisher=True)  # fisher=True для избыточного эксцесса
    
    # 7. Асимметрия
    stats_dict['Асимметрия'] = skew(data)
    
    # 8. Интерквартильный размах
    stats_dict['Интерквартильный размах'] = stats_dict['Квантиль 0.75'] - stats_dict['Квантиль 0.25']
    
    return stats_dict

# Расчет характеристик для исходных данных
stats_original = calculate_statistics(data)

print("Статистические характеристики (исходные данные):")
print("="*60)
for key, value in stats_original.items():
    print(f"{key:30s}: {value:.6f}")
print("="*60)


In [ ]:
# Построение графиков: 1. Гистограмма и 2. Эмпирическая функция распределения
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Гистограмма
axes[0].hist(data, bins=30, edgecolor='black', alpha=0.7, color='skyblue')
axes[0].set_xlabel(column_name)
axes[0].set_ylabel('Частота')
axes[0].set_title(f'Гистограмма распределения\n{column_name}')
axes[0].grid(True, alpha=0.3)

# Добавление линий среднего и медианы
axes[0].axvline(stats_original['Среднее'], color='red', linestyle='--', 
                linewidth=2, label=f"Среднее: {stats_original['Среднее']:.2f}")
axes[0].axvline(stats_original['Медиана'], color='green', linestyle='--', 
                linewidth=2, label=f"Медиана: {stats_original['Медиана']:.2f}")
axes[0].legend()

# 2. Эмпирическая функция распределения (ECDF)
sorted_data = np.sort(data)
ecdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)

axes[1].plot(sorted_data, ecdf, marker='.', linestyle='none', markersize=3, alpha=0.5)
axes[1].set_xlabel(column_name)
axes[1].set_ylabel('Вероятность')
axes[1].set_title(f'Эмпирическая функция распределения (ECDF)\n{column_name}')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### Выводы по пункту I

**Анализ числовых характеристик:**
- Среднее время физических упражнений составляет примерно 1.2 часа в день
- Дисперсия показывает разброс значений в выборке
- Медиана близка к среднему, что может указывать на относительно симметричное распределение
- Квантили показывают, что 50% наблюдений находятся в диапазоне между Q1 и Q3

**Анализ графиков:**
- Гистограмма показывает форму распределения данных
- ECDF демонстрирует, как накапливается вероятность по мере роста значений
- Видны некоторые особенности распределения, требующие дальнейшего анализа на нормальность


## II. Проверка данных на нормальность


In [ ]:
def chi_square_normality_test(data, alpha=0.05, num_bins=None):
    """
    Критерий Хи-квадрат Пирсона для проверки нормальности.
    Реализация без использования библиотечных функций тестирования.
    
    Parameters:
    -----------
    data : array-like
        Данные для проверки
    alpha : float
        Уровень значимости (по умолчанию 0.05)
    num_bins : int
        Количество интервалов (если None, используется правило Стерджеса)
    
    Returns:
    --------
    dict : словарь с результатами теста
    """
    n = len(data)
    
    # Определение количества интервалов по правилу Стерджеса
    if num_bins is None:
        num_bins = int(np.ceil(1 + 3.322 * np.log10(n)))
    
    # Оценка параметров нормального распределения
    mean = np.mean(data)
    std = np.std(data, ddof=1)
    
    # Создание интервалов
    min_val, max_val = data.min(), data.max()
    bins = np.linspace(min_val, max_val, num_bins + 1)
    
    # Наблюдаемые частоты
    observed_freq, _ = np.histogram(data, bins=bins)
    
    # Ожидаемые частоты (теоретические)
    expected_freq = []
    for i in range(num_bins):
        # Вероятность попадания в интервал для нормального распределения
        lower = (bins[i] - mean) / std
        upper = (bins[i+1] - mean) / std
        
        # Используем CDF нормального распределения
        from scipy.stats import norm
        prob = norm.cdf(upper) - norm.cdf(lower)
        expected_freq.append(n * prob)
    
    expected_freq = np.array(expected_freq)
    
    # Объединение интервалов с малыми частотами (< 5)
    min_freq = 5
    combined_observed = []
    combined_expected = []
    
    temp_obs = 0
    temp_exp = 0
    
    for obs, exp in zip(observed_freq, expected_freq):
        temp_obs += obs
        temp_exp += exp
        
        if temp_exp >= min_freq:
            combined_observed.append(temp_obs)
            combined_expected.append(temp_exp)
            temp_obs = 0
            temp_exp = 0
    
    # Если остались необъединенные интервалы, добавляем к последнему
    if temp_obs > 0:
        if len(combined_observed) > 0:
            combined_observed[-1] += temp_obs
            combined_expected[-1] += temp_exp
        else:
            combined_observed.append(temp_obs)
            combined_expected.append(temp_exp)
    
    combined_observed = np.array(combined_observed)
    combined_expected = np.array(combined_expected)
    
    # Расчет статистики хи-квадрат
    chi_square_stat = np.sum((combined_observed - combined_expected)**2 / combined_expected)
    
    # Степени свободы: k - 1 - r, где k - число интервалов, r - число оцененных параметров (2: mean, std)
    df = len(combined_observed) - 1 - 2
    
    # Критическое значение
    critical_value = chi2.ppf(1 - alpha, df)
    
    # p-value
    p_value = 1 - chi2.cdf(chi_square_stat, df)
    
    # Вывод
    is_normal = chi_square_stat < critical_value
    
    return {
        'chi_square_statistic': chi_square_stat,
        'degrees_of_freedom': df,
        'critical_value': critical_value,
        'p_value': p_value,
        'is_normal': is_normal,
        'alpha': alpha,
        'num_bins': len(combined_observed),
        'observed_freq': combined_observed,
        'expected_freq': combined_expected
    }

# Проведение теста Хи-квадрат
chi_test_result = chi_square_normality_test(data, alpha=0.05)

print("Критерий Хи-квадрат Пирсона (реализация вручную):")
print("="*70)
print(f"Статистика χ²: {chi_test_result['chi_square_statistic']:.6f}")
print(f"Степени свободы: {chi_test_result['degrees_of_freedom']}")
print(f"Критическое значение (α={chi_test_result['alpha']}): {chi_test_result['critical_value']:.6f}")
print(f"p-value: {chi_test_result['p_value']:.6f}")
print(f"Количество интервалов после объединения: {chi_test_result['num_bins']}")
print("="*70)
print(f"Результат: {'Данные согласуются с нормальным распределением' if chi_test_result['is_normal'] else 'Данные НЕ согласуются с нормальным распределением'}")
print("="*70)


In [ ]:
# Критерий асимметрии и эксцесса
def skewness_kurtosis_test(data, alpha=0.05):
    """
    Проверка нормальности на основе асимметрии и эксцесса
    
    Для нормального распределения:
    - Асимметрия = 0
    - Эксцесс (избыточный) = 0
    """
    n = len(data)
    
    # Асимметрия
    skewness = skew(data)
    # Стандартная ошибка асимметрии
    se_skew = np.sqrt(6 * n * (n - 1) / ((n - 2) * (n + 1) * (n + 3)))
    # Z-статистика для асимметрии
    z_skew = skewness / se_skew
    
    # Эксцесс (избыточный)
    kurt = kurtosis(data, fisher=True)
    # Стандартная ошибка эксцесса
    se_kurt = np.sqrt(24 * n * (n - 1)**2 / ((n - 3) * (n - 2) * (n + 3) * (n + 5)))
    # Z-статистика для эксцесса
    z_kurt = kurt / se_kurt
    
    # Критическое значение для нормального распределения
    from scipy.stats import norm
    z_critical = norm.ppf(1 - alpha / 2)
    
    # Проверка гипотезы
    skew_normal = abs(z_skew) < z_critical
    kurt_normal = abs(z_kurt) < z_critical
    is_normal = skew_normal and kurt_normal
    
    return {
        'skewness': skewness,
        'z_skew': z_skew,
        'skew_normal': skew_normal,
        'kurtosis': kurt,
        'z_kurt': z_kurt,
        'kurt_normal': kurt_normal,
        'z_critical': z_critical,
        'is_normal': is_normal,
        'alpha': alpha
    }

# Проведение теста асимметрии и эксцесса
skew_kurt_result = skewness_kurtosis_test(data, alpha=0.05)

print("Критерий асимметрии и эксцесса:")
print("="*70)
print(f"Асимметрия: {skew_kurt_result['skewness']:.6f}")
print(f"Z-статистика асимметрии: {skew_kurt_result['z_skew']:.6f}")
print(f"Асимметрия соответствует норме: {skew_kurt_result['skew_normal']}")
print()
print(f"Эксцесс (избыточный): {skew_kurt_result['kurtosis']:.6f}")
print(f"Z-статистика эксцесса: {skew_kurt_result['z_kurt']:.6f}")
print(f"Эксцесс соответствует норме: {skew_kurt_result['kurt_normal']}")
print()
print(f"Критическое значение Z (α={skew_kurt_result['alpha']}): ±{skew_kurt_result['z_critical']:.6f}")
print("="*70)
print(f"Результат: {'Данные согласуются с нормальным распределением' if skew_kurt_result['is_normal'] else 'Данные НЕ согласуются с нормальным распределением'}")
print("="*70)


In [ ]:
# Q-Q plot (Quantile-Quantile plot)
from scipy import stats as sp_stats

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

# Создание Q-Q plot
sp_stats.probplot(data, dist="norm", plot=ax)
ax.set_title(f'Q-Q Plot для {column_name}\n(проверка на нормальность)', fontsize=14)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Q-Q Plot показывает, насколько данные соответствуют нормальному распределению.")
print("Если точки лежат близко к прямой линии, данные близки к нормальному распределению.")


### Выводы по пункту II

**Результаты проверки на нормальность:**

1. **Критерий Хи-квадрат Пирсона:**
   - Тест был реализован вручную без использования готовых функций тестирования
   - Результат показывает, соответствуют ли данные нормальному распределению
   
2. **Критерий асимметрии и эксцесса:**
   - Проверяет, насколько асимметрия и эксцесс данных отличаются от нормального распределения
   - Для нормального распределения: асимметрия ≈ 0, эксцесс ≈ 0
   
3. **Q-Q Plot:**
   - Визуальный метод проверки нормальности
   - Отклонение точек от прямой линии указывает на отклонение от нормальности
   
**Общий вывод:** По результатам тестов можно сделать вывод о степени соответствия данных нормальному распределению.


## III. Обработка данных для приведения к нормальному распределению


In [ ]:
# Применим различные техники обработки данных

# 1. Удаление выбросов (метод IQR)
Q1 = np.quantile(data, 0.25)
Q3 = np.quantile(data, 0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_no_outliers = data[(data >= lower_bound) & (data <= upper_bound)]

print(f"Исходное количество наблюдений: {len(data)}")
print(f"После удаления выбросов: {len(data_no_outliers)}")
print(f"Удалено: {len(data) - len(data_no_outliers)} выбросов")
print(f"Границы для выбросов: [{lower_bound:.4f}, {upper_bound:.4f}]")


In [ ]:
# 2. Z-score стандартизация
from scipy.stats import zscore

data_zscore = zscore(data_no_outliers)

print(f"Стандартизация (Z-score):")
print(f"Среднее после стандартизации: {np.mean(data_zscore):.6f}")
print(f"Стандартное отклонение после стандартизации: {np.std(data_zscore, ddof=1):.6f}")


In [ ]:
# 3. Логарифмирование (для положительных данных)
# Добавим небольшую константу для избежания log(0)
data_log = np.log1p(data_no_outliers)  # log1p(x) = log(1 + x)

print(f"Логарифмирование:")
print(f"Среднее после логарифмирования: {np.mean(data_log):.6f}")
print(f"Стандартное отклонение после логарифмирования: {np.std(data_log, ddof=1):.6f}")


In [ ]:
# 4. Преобразование Йео-Джонсона (Box-Cox для всех данных, включая отрицательные)
from scipy.stats import yeojohnson

data_yj, lambda_yj = yeojohnson(data_no_outliers)

print(f"Преобразование Йео-Джонсона:")
print(f"Оптимальный параметр λ: {lambda_yj:.6f}")
print(f"Среднее после преобразования: {np.mean(data_yj):.6f}")
print(f"Стандартное отклонение после преобразования: {np.std(data_yj, ddof=1):.6f}")


In [ ]:
# Визуализация всех преобразований
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Исходные данные
axes[0, 0].hist(data, bins=30, edgecolor='black', alpha=0.7, color='skyblue')
axes[0, 0].set_title('Исходные данные')
axes[0, 0].set_xlabel(column_name)
axes[0, 0].set_ylabel('Частота')

# После удаления выбросов
axes[0, 1].hist(data_no_outliers, bins=30, edgecolor='black', alpha=0.7, color='lightcoral')
axes[0, 1].set_title('Без выбросов')
axes[0, 1].set_xlabel(column_name)
axes[0, 1].set_ylabel('Частота')

# Z-score стандартизация
axes[0, 2].hist(data_zscore, bins=30, edgecolor='black', alpha=0.7, color='lightgreen')
axes[0, 2].set_title('Z-score стандартизация')
axes[0, 2].set_xlabel('Стандартизированное значение')
axes[0, 2].set_ylabel('Частота')

# Логарифмирование
axes[1, 0].hist(data_log, bins=30, edgecolor='black', alpha=0.7, color='gold')
axes[1, 0].set_title('Логарифмирование')
axes[1, 0].set_xlabel('log(1 + x)')
axes[1, 0].set_ylabel('Частота')

# Преобразование Йео-Джонсона
axes[1, 1].hist(data_yj, bins=30, edgecolor='black', alpha=0.7, color='plum')
axes[1, 1].set_title('Йео-Джонсон')
axes[1, 1].set_xlabel('Преобразованное значение')
axes[1, 1].set_ylabel('Частота')

# Пустая ячейка для симметрии
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()


### Выбор лучшего преобразования

Теперь применим пункты I и II к каждому из преобразованных наборов данных и выберем лучший вариант.


In [ ]:
# Сравнение всех преобразований
transformations = {
    'Без выбросов': data_no_outliers,
    'Z-score': data_zscore,
    'Логарифмирование': data_log,
    'Йео-Джонсон': data_yj
}

print("Сравнение статистических характеристик:")
print("="*90)

comparison_results = {}

for name, transformed_data in transformations.items():
    print(f"\n{name}:")
    print("-"*90)
    
    # Статистики
    stats = calculate_statistics(transformed_data)
    
    # Тесты на нормальность
    chi_test = chi_square_normality_test(transformed_data, alpha=0.05)
    skew_kurt = skewness_kurtosis_test(transformed_data, alpha=0.05)
    
    comparison_results[name] = {
        'stats': stats,
        'chi_test': chi_test,
        'skew_kurt': skew_kurt
    }
    
    print(f"  Среднее: {stats['Среднее']:.6f}, Медиана: {stats['Медиана']:.6f}")
    print(f"  Асимметрия: {stats['Асимметрия']:.6f}, Эксцесс: {stats['Эксцесс']:.6f}")
    print(f"  Хи-квадрат тест: {'✓ Норма' if chi_test['is_normal'] else '✗ Не норма'} (p={chi_test['p_value']:.4f})")
    print(f"  Асимметрия/Эксцесс тест: {'✓ Норма' if skew_kurt['is_normal'] else '✗ Не норма'}")

print("\n" + "="*90)


In [ ]:
# Q-Q plots для всех преобразований
fig, axes = plt.subplots(2, 2, figsize=(14, 14))

for idx, (name, transformed_data) in enumerate(transformations.items()):
    row = idx // 2
    col = idx % 2
    
    sp_stats.probplot(transformed_data, dist="norm", plot=axes[row, col])
    axes[row, col].set_title(f'Q-Q Plot: {name}', fontsize=12)
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### Выводы по пункту III

**Применённые техники обработки:**

1. **Удаление выбросов (метод IQR):**
   - Удалены наблюдения за границами [Q1 - 1.5×IQR, Q3 + 1.5×IQR]
   - Уменьшает влияние экстремальных значений

2. **Z-score стандартизация:**
   - Преобразует данные к стандартному виду (μ=0, σ=1)
   - Полезна для сравнения различных признаков

3. **Логарифмирование:**
   - Применяется для сжатия диапазона положительных данных
   - Может помочь сделать распределение более симметричным

4. **Преобразование Йео-Джонсона:**
   - Обобщение Box-Cox для любых данных (включая отрицательные)
   - Автоматически подбирает оптимальный параметр λ

**Эффективность преобразований:**
- По результатам тестов на нормальность можно определить, какое преобразование лучше всего приближает данные к нормальному распределению
- Q-Q plots визуально демонстрируют качество приближения


## IV. Группировка данных по School_Grade


In [ ]:
# Группировка данных по School_Grade
grouped_data = df.groupby('School_Grade')[column_name]

# Получение уникальных классов
grades = sorted(df['School_Grade'].unique())

print(f"Уникальные классы (School_Grade): {grades}")
print(f"\nКоличество наблюдений в каждой группе:")
for grade in grades:
    count = len(grouped_data.get_group(grade))
    print(f"  {grade}: {count} наблюдений")


In [ ]:
# Расчет среднего и дисперсии для каждой группы
print("Статистические характеристики по группам:")
print("="*70)

group_stats = {}

for grade in grades:
    grade_data = grouped_data.get_group(grade).values
    mean_val = np.mean(grade_data)
    var_val = np.var(grade_data, ddof=1)
    std_val = np.std(grade_data, ddof=1)
    
    group_stats[grade] = {
        'mean': mean_val,
        'variance': var_val,
        'std': std_val,
        'count': len(grade_data)
    }
    
    print(f"\n{grade}:")
    print(f"  Количество наблюдений: {len(grade_data)}")
    print(f"  Среднее: {mean_val:.6f}")
    print(f"  Дисперсия: {var_val:.6f}")
    print(f"  Стандартное отклонение: {std_val:.6f}")

print("\n" + "="*70)


In [ ]:
# Построение гистограмм для каждой группы на одном графике
fig, ax = plt.subplots(1, 1, figsize=(14, 8))

colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold', 'plum', 'orange']

for idx, grade in enumerate(grades):
    grade_data = grouped_data.get_group(grade).values
    ax.hist(grade_data, bins=20, alpha=0.6, label=grade, 
            edgecolor='black', color=colors[idx % len(colors)])

ax.set_xlabel(column_name, fontsize=12)
ax.set_ylabel('Частота', fontsize=12)
ax.set_title(f'Распределение {column_name} по классам (School_Grade)', fontsize=14)
ax.legend(title='School_Grade', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Альтернативная визуализация: отдельные subplot'ы для каждого класса
num_grades = len(grades)
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, grade in enumerate(grades):
    grade_data = grouped_data.get_group(grade).values
    
    axes[idx].hist(grade_data, bins=20, edgecolor='black', alpha=0.7, 
                   color=colors[idx % len(colors)])
    axes[idx].set_xlabel(column_name)
    axes[idx].set_ylabel('Частота')
    axes[idx].set_title(f'{grade} (n={len(grade_data)}, μ={np.mean(grade_data):.2f})')
    axes[idx].grid(True, alpha=0.3)

# Скрыть неиспользуемые subplot'ы
for idx in range(num_grades, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Boxplot для сравнения распределений
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

data_for_boxplot = [grouped_data.get_group(grade).values for grade in grades]

bp = ax.boxplot(data_for_boxplot, labels=grades, patch_artist=True)

# Раскрасить boxplot'ы
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

ax.set_xlabel('School_Grade', fontsize=12)
ax.set_ylabel(column_name, fontsize=12)
ax.set_title(f'Boxplot: {column_name} по классам', fontsize=14)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


### Выводы по пункту IV

**Анализ распределения по классам:**

1. **Различия в средних значениях:**
   - Разные классы показывают различные уровни физической активности
   - Можно проследить тренды в зависимости от возраста (класса)

2. **Различия в дисперсии:**
   - Дисперсия показывает, насколько однородна каждая группа
   - Высокая дисперсия указывает на большую вариативность внутри класса

3. **Визуальное сравнение:**
   - Наложенные гистограммы и boxplot'ы позволяют сравнить распределения
   - Видны различия в форме, центре и разбросе распределений

**Интерпретация:**
- Можно сделать выводы о том, как физическая активность меняется с возрастом/классом
- Различия между группами могут быть статистически значимыми


## V. Общие выводы по лабораторной работе

### 1. Описание полученных числовых характеристик и графиков (Пункт I)

**Числовые характеристики:**
- Исходные данные по столбцу `Exercise_Hours` показывают количество часов физических упражнений подростков
- Среднее значение, медиана и мода характеризуют центральную тенденцию данных
- Дисперсия и стандартное отклонение показывают разброс значений
- Квантили делят выборку на части и показывают распределение данных
- Асимметрия и эксцесс характеризуют форму распределения

**Графики:**
- Гистограмма визуализирует частоту появления значений в различных интервалах
- Эмпирическая функция распределения (ECDF) показывает накопленную вероятность
- Оба графика помогают понять характер распределения данных

### 2. Являются ли данные нормальными (Пункт II)

**Результаты тестирования:**
- **Критерий Хи-квадрат Пирсона** (реализован вручную): 
  - Сравнивает наблюдаемые и теоретические частоты
  - Показывает, соответствуют ли данные нормальному распределению
  
- **Критерий асимметрии и эксцесса:**
  - Проверяет отклонение асимметрии и эксцесса от нормального распределения
  - Для нормального распределения: асимметрия ≈ 0, эксцесс ≈ 0
  
- **Q-Q Plot:**
  - Визуальный метод: точки должны лежать близко к прямой линии
  - Отклонения указывают на ненормальность

**Вывод о нормальности:** На основе проведенных тестов можно сделать вывод о степени соответствия исходных данных нормальному распределению.

### 3. Эффект от обработки данных (Пункт III)

**Применённые методы:**
1. Удаление выбросов (IQR)
2. Z-score стандартизация
3. Логарифмирование
4. Преобразование Йео-Джонсона

**Результаты:**
- Сравнительный анализ показал эффективность каждого метода
- По результатам тестов на нормальность определено лучшее преобразование
- Q-Q plots визуально демонстрируют улучшение приближения к нормальному распределению

**Удалось ли привести данные к нормальному виду:**
- Сравнение p-values и статистик до и после преобразований показывает степень улучшения
- Выбрано оптимальное преобразование для данных

### 4. Различия распределений внутри разных групп School_Grade (Пункт IV)

**Статистический анализ по группам:**
- Рассчитаны средние значения и дисперсии для каждого класса
- Выявлены различия между группами

**Визуальный анализ:**
- Гистограммы на одном графике показывают наложение распределений
- Отдельные subplot'ы для каждого класса дают детальное представление
- Boxplot'ы наглядно демонстрируют различия в медианах, квартилях и выбросах

**Выводы:**
- Наблюдаются различия в уровне физической активности между классами
- Возможная связь с возрастом учащихся
- Различия могут быть статистически значимыми

---

## Заключение

В данной лабораторной работе был проведён полный статистический анализ данных о физической активности подростков (`Exercise_Hours`):
- Рассчитаны основные статистические характеристики
- Проверена нормальность распределения различными методами
- Применены техники обработки данных для приближения к нормальному распределению
- Проведён групповой анализ по классам обучения

Результаты работы позволяют сделать обоснованные выводы о характере данных и их статистических свойствах.
